In [1]:
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import dlib
from PIL import Image
import joblib

from prediction import *

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [3]:
train = ReadDibCsv(100, './data2/train/')
test = ReadDibCsv(100, './data2/test/')

In [4]:
train_level = np.zeros((100, 4))
cnt = len(train)
eye_list = [43,44,46,47]  ## 왼쪽 눈
mouth_list =[61, 62, 63, 65, 66, 67]   ## 안쪽 입 중앙
for i in range(cnt):
    x = train[i,:,0]
    y = train[i,:,1]
    
    minx = min(x)
    miny = min(y)
    maxy = max(y)
    
    height = max(y) - min(y)
    
    x = (x - minx) / height
    y = (y - miny) / height
    
    eye_center = y[eye_list].mean()
    mouth_center = y[mouth_list].mean()
    
    train_level[i][0] = eye_center
    train_level[i][1] = y[33]
    train_level[i][2] = mouth_center
    train_level[i][3] = y[8]

In [5]:
test_level = np.zeros((100, 4))
cnt = len(test)
eye_list = [43,44,46,47]
mouth_list =[61, 62, 63, 65, 66, 67]
for i in range(cnt):
    x = test[i,:,0]
    y = test[i,:,1]
    
    minx = min(x)
    miny = min(y)
    maxy = max(y)
    
    height = max(y) - min(y)
    
    x = (x - minx) / height
    y = (y - miny) / height
    
    eye_center = y[eye_list].mean()
    mouth_center = y[mouth_list].mean()
    
    test_level[i][0] = eye_center
    test_level[i][1] = y[33]
    test_level[i][2] = mouth_center
    test_level[i][3] = y[8]

In [6]:
train_y = pd.read_csv('./DataSet2/Train/csv/train.csv')
train_y = train_y[['eyeLevel', 'mouthLevel', 'noseLevel', 'chinLevel']]
test_y = pd.read_csv('./DataSet2/Test/csv/test.csv')
test_y = test_y[['eyeLevel', 'mouthLevel', 'noseLevel', 'chinLevel']]

In [7]:
model = LinearRegression()
model.fit(train_level, train_y)
pred = model.predict(test_level)
df = pd.DataFrame(pred)
df.columns = ['eyeLevel','mouthLevel','noseLevel','chinLevel']
df

,eyeLevel,mouthLevel,noseLevel,chinLevel
0,0.061087,-0.007866,0.013161,0.007878
1,0.056171,-0.005207,0.016029,0.006740
2,0.058795,-0.002312,0.014073,0.003780
3,0.059149,-0.008033,0.017054,0.007113
4,0.061609,-0.007465,0.013714,0.006971
...,...,...,...,...
95,0.057283,-0.006997,0.024755,0.003351
96,0.055730,-0.002376,0.019222,0.002899
97,0.054991,-0.002683,0.024596,0.000876
98,0.055505,-0.000904,0.018509,0.002086


In [8]:
test_y

,eyeLevel,mouthLevel,noseLevel,chinLevel
0,0.062867,-0.008600,0.011012,0.005435
1,0.054299,-0.004324,0.015606,0.006090
2,0.060204,-0.000589,0.017270,0.007846
3,0.058193,-0.008047,0.017450,0.005343
4,0.061076,-0.006287,0.014368,0.010111
...,...,...,...,...
95,0.055513,-0.008897,0.024248,0.000987
96,0.055671,-0.001439,0.016712,-0.001717
97,0.053516,-0.003197,0.025003,0.001162
98,0.059978,0.001534,0.019074,0.003165


In [9]:
mae = mean_absolute_error(test_y, pred)
mae

0.00201619814208174

In [10]:
e_min = 0.05098964
e_max = 0.06300907
n_min = 0.0108679
n_max = 0.02551717
m_min = -0.008652932
m_max = 0.001820078
j_min = -0.001825406
j_max = 0.01026576 

In [11]:
df['eyeLevel'] = (df['eyeLevel'] - e_min) / (e_max - e_min) 
df['noseLevel'] = (df['noseLevel'] - n_min) / (n_max - n_min) 
df['mouthLevel'] = (df['mouthLevel'] - m_min) / (m_max - m_min) 
df['chinLevel'] = (df['chinLevel'] - j_min) / (j_max - j_min) 

In [12]:
df

,eyeLevel,mouthLevel,noseLevel,chinLevel
0,0.840075,0.075170,0.156560,0.802485
1,0.431070,0.329070,0.352342,0.708394
2,0.649390,0.605481,0.218821,0.463571
3,0.678882,0.059192,0.422248,0.739262
4,0.883549,0.113424,0.194264,0.727527
...,...,...,...,...
95,0.523625,0.158133,0.947990,0.428140
96,0.394368,0.599309,0.570281,0.390773
97,0.332916,0.570015,0.937132,0.223408
98,0.375689,0.739876,0.521576,0.323520


In [13]:
pos_y = pd.read_csv('./DataSet2/Test/csv/test.csv')
pos_y = pos_y[['eyePosition','mouthPosition','nosePosition','chinPosition']]

In [14]:
pos_y

,eyePosition,mouthPosition,nosePosition,chinPosition
0,0.987580,0.083063,0.011575,0.444558
1,0.263217,0.388587,0.380518,0.798688
2,0.755307,0.762916,0.510423,0.498208
3,0.579981,0.057911,0.521110,0.861411
4,0.831355,0.231283,0.281028,0.921561
...,...,...,...,...
95,0.359737,0.046782,0.924631,0.225515
96,0.372337,0.659678,0.469275,0.155858
97,0.200910,0.536925,0.969470,0.492540
98,0.735607,0.938573,0.617494,0.707767


In [15]:
mae = mean_absolute_error(pos_y['mouthPosition'], df['mouthLevel'])
mae

0.11009140749913254

In [16]:
mae = mean_absolute_error(pos_y['nosePosition'], df['noseLevel'])
mae

0.15608784873192905

In [17]:
mae = mean_absolute_error(pos_y['eyePosition'], df['eyeLevel'])
mae

0.16147491510449388

In [18]:
mae = mean_absolute_error(pos_y['chinPosition'], df['chinLevel'])
mae

0.1833912524397861

In [19]:
mae = mean_absolute_error(pos_y, df)
mae

0.1527613559438354